In [1]:
##########################################################################################################
# VERSION  	DESARROLLADOR             FECHA        DESCRIPCION
# -------------------------------------------------------------
#  1        Walter Albites Azarte     10/12/2021   Curso PySpark Entorno Local - Dataframe
##########################################################################################################

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.1.2-bin-hadoop2.7'

In [3]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf= pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark=SparkSession(sc)

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [5]:
### Crear Dataframe Spark personas
schemaPersona = StructType([
    StructField("dni", StringType(),True),
    StructField("nombre", StringType(),True),
    StructField("edad", IntegerType(),True)
])

dataPersona = ([("99999999", "Walter",35),
                ("88888888", "Susan",30),
                ("77777777", "Alejandro",12),
                ("66666666", "Pedro",30),
                ("55555555", "Karina",35),
                ("44444444", "Andy",26),
                ("33333333", "Magaly",45),
                ("22222222", "Gian",28),
                ("11111111", "Raul",45),
                ("00000000", "Elena",40),
                ("10101010", None, None),
                ("20202020",'',None)])

df_personas=spark.createDataFrame(dataPersona,schema=schemaPersona)
df_personas.show()

+--------+---------+----+
|     dni|   nombre|edad|
+--------+---------+----+
|99999999|   Walter|  35|
|88888888|    Susan|  30|
|77777777|Alejandro|  12|
|66666666|    Pedro|  30|
|55555555|   Karina|  35|
|44444444|     Andy|  26|
|33333333|   Magaly|  45|
|22222222|     Gian|  28|
|11111111|     Raul|  45|
|00000000|    Elena|  40|
|10101010|     null|null|
|20202020|         |null|
+--------+---------+----+



In [6]:
df_personas.printSchema()

root
 |-- dni: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: integer (nullable = true)



In [7]:
# concat_ws() función de Pyspark concatena múltiples columnas de cadenas en una sola columna con un separador
# o delimitador determinado.
df_personas.withColumn('datos_personales', concat_ws(',',col('dni'),col('nombre'))).show(5,False)

+--------+---------+----+------------------+
|dni     |nombre   |edad|datos_personales  |
+--------+---------+----+------------------+
|99999999|Walter   |35  |99999999,Walter   |
|88888888|Susan    |30  |88888888,Susan    |
|77777777|Alejandro|12  |77777777,Alejandro|
|66666666|Pedro    |30  |66666666,Pedro    |
|55555555|Karina   |35  |55555555,Karina   |
+--------+---------+----+------------------+
only showing top 5 rows



In [8]:
# Coalesce función de Pyspark Devuelve la primera columna que no es nula.
df_documentoidentidad = spark.createDataFrame([(None, None), (88888888, None), (None, 99999999)], ("dni_1", "dni_2"))
df_documentoidentidad.show()

+--------+--------+
|   dni_1|   dni_2|
+--------+--------+
|    null|    null|
|88888888|    null|
|    null|99999999|
+--------+--------+



In [9]:
df_documentoidentidad.select(coalesce(col("dni_1"), col("dni_2")).alias("dni")).show()

+--------+
|     dni|
+--------+
|    null|
|88888888|
|99999999|
+--------+



In [10]:
df_personas.show()

+--------+---------+----+
|     dni|   nombre|edad|
+--------+---------+----+
|99999999|   Walter|  35|
|88888888|    Susan|  30|
|77777777|Alejandro|  12|
|66666666|    Pedro|  30|
|55555555|   Karina|  35|
|44444444|     Andy|  26|
|33333333|   Magaly|  45|
|22222222|     Gian|  28|
|11111111|     Raul|  45|
|00000000|    Elena|  40|
|10101010|     null|null|
|20202020|         |null|
+--------+---------+----+



In [11]:
### Ejemplos utilizando UDF

In [12]:
#Crear una funcion que convierta en minusculas
def ConvertirMinusculas(texto):
    texto=texto.lower()
    return texto

In [13]:
# Crear un UDF a apartir de una funcion de Python
udf_convertir_misnusculas=udf(ConvertirMinusculas,StringType())

In [14]:
# Crear una columna normalizada en minuscula con la columna nombre y curso, Saldra error al parecer lee todas las filas
# del dataframe Los null y vacios por mas que se use la condicional

'''
df_personasy=df_personas.withColumn("nombre_normalizado",
                                    when(((col("nombre").isNotNull()) & (col("nombre")!="")),
                                         udf_convertir_misnusculas("nombre")).\
                                    otherwise("")
                                   )

df_personasy.show()
'''

# AttributeError: 'NoneType' object has no attribute 'lower'

'\ndf_personasy=df_personas.withColumn("nombre_normalizado",\n                                    when(((col("nombre").isNotNull()) & (col("nombre")!="")),\n                                         udf_convertir_misnusculas("nombre")).                                    otherwise("")\n                                   )\n\ndf_personasy.show()\n'

In [15]:
#Solucion 1
#Crear una nueva columna temporal para colocar valores vacios a los NULL
df_personasz=df_personas.select('*',col("nombre").alias("nombre_temp"))
df_personasz=df_personasz.na.fill(value="",subset=["nombre_temp"])
df_personasz.show()

+--------+---------+----+-----------+
|     dni|   nombre|edad|nombre_temp|
+--------+---------+----+-----------+
|99999999|   Walter|  35|     Walter|
|88888888|    Susan|  30|      Susan|
|77777777|Alejandro|  12|  Alejandro|
|66666666|    Pedro|  30|      Pedro|
|55555555|   Karina|  35|     Karina|
|44444444|     Andy|  26|       Andy|
|33333333|   Magaly|  45|     Magaly|
|22222222|     Gian|  28|       Gian|
|11111111|     Raul|  45|       Raul|
|00000000|    Elena|  40|      Elena|
|10101010|     null|null|           |
|20202020|         |null|           |
+--------+---------+----+-----------+



In [16]:
# Aplicamos el filtro y utilizamos el UDF 
df_personasz=df_personasz.withColumn("nombre_normalizado",
                                    when(col("nombre_temp")!="",
                                         udf_convertir_misnusculas("nombre_temp")).\
                                         #lit("Con Dato")).
                                    otherwise("")
                                   ).drop("nombre_temp")
df_personasz.show(20,False)

+--------+---------+----+------------------+
|dni     |nombre   |edad|nombre_normalizado|
+--------+---------+----+------------------+
|99999999|Walter   |35  |walter            |
|88888888|Susan    |30  |susan             |
|77777777|Alejandro|12  |alejandro         |
|66666666|Pedro    |30  |pedro             |
|55555555|Karina   |35  |karina            |
|44444444|Andy     |26  |andy              |
|33333333|Magaly   |45  |magaly            |
|22222222|Gian     |28  |gian              |
|11111111|Raul     |45  |raul              |
|00000000|Elena    |40  |elena             |
|10101010|null     |null|                  |
|20202020|         |null|                  |
+--------+---------+----+------------------+



In [17]:
#Solucion 2
#Modificar la funcion cuando es None asignarle vacio
def ConvertirMinusculas(texto):
    if texto==None:
        texto=""
    else:
        texto=texto.lower()
    return texto

udf_convertir_misnusculas=udf(ConvertirMinusculas,StringType())

In [18]:
df_personasm=df_personas.withColumn("nombre_normalizado",
                                    when(((col("nombre").isNotNull()) & (col("nombre")!="")),
                                         udf_convertir_misnusculas("nombre")).\
                                    otherwise("")
                                   )
df_personasm.show(20,False)

+--------+---------+----+------------------+
|dni     |nombre   |edad|nombre_normalizado|
+--------+---------+----+------------------+
|99999999|Walter   |35  |walter            |
|88888888|Susan    |30  |susan             |
|77777777|Alejandro|12  |alejandro         |
|66666666|Pedro    |30  |pedro             |
|55555555|Karina   |35  |karina            |
|44444444|Andy     |26  |andy              |
|33333333|Magaly   |45  |magaly            |
|22222222|Gian     |28  |gian              |
|11111111|Raul     |45  |raul              |
|00000000|Elena    |40  |elena             |
|10101010|null     |null|                  |
|20202020|         |null|                  |
+--------+---------+----+------------------+



In [19]:
df_personas_x=df_personas

In [20]:
# Renombrar Columnas utilizando withColumnRenamed, Consume mucha memoria
for column in df_personas_x.columns:
        df_personas_x=df_personas_x.withColumnRenamed(column,'personas_' + column)

In [21]:
df_personas_x.show(5,False)

+------------+---------------+-------------+
|personas_dni|personas_nombre|personas_edad|
+------------+---------------+-------------+
|99999999    |Walter         |35           |
|88888888    |Susan          |30           |
|77777777    |Alejandro      |12           |
|66666666    |Pedro          |30           |
|55555555    |Karina         |35           |
+------------+---------------+-------------+
only showing top 5 rows



In [22]:
# Renombrar columnas utilizando Alias, no consume mucha memoria Recomendable
df_personas_y=df_personas.select([col(i).alias('persons_'+i) for i in df_personas.columns])
df_personas_y.show()

+-----------+--------------+------------+
|persons_dni|persons_nombre|persons_edad|
+-----------+--------------+------------+
|   99999999|        Walter|          35|
|   88888888|         Susan|          30|
|   77777777|     Alejandro|          12|
|   66666666|         Pedro|          30|
|   55555555|        Karina|          35|
|   44444444|          Andy|          26|
|   33333333|        Magaly|          45|
|   22222222|          Gian|          28|
|   11111111|          Raul|          45|
|   00000000|         Elena|          40|
|   10101010|          null|        null|
|   20202020|              |        null|
+-----------+--------------+------------+



In [23]:
# Indentificar NULL, Nan y Vacios
df_personas.select([count(when( (isnan(c))| (col(c).isNull()) | (col(c)==''), c)).alias(c) for c in df_personas.columns]).show()

+---+------+----+
|dni|nombre|edad|
+---+------+----+
|  0|     2|   2|
+---+------+----+



In [24]:
# Adicionar una columna con la fecha del sistema
df_personas.withColumn('fecha',current_date()).show()

+--------+---------+----+----------+
|     dni|   nombre|edad|     fecha|
+--------+---------+----+----------+
|99999999|   Walter|  35|2022-05-19|
|88888888|    Susan|  30|2022-05-19|
|77777777|Alejandro|  12|2022-05-19|
|66666666|    Pedro|  30|2022-05-19|
|55555555|   Karina|  35|2022-05-19|
|44444444|     Andy|  26|2022-05-19|
|33333333|   Magaly|  45|2022-05-19|
|22222222|     Gian|  28|2022-05-19|
|11111111|     Raul|  45|2022-05-19|
|00000000|    Elena|  40|2022-05-19|
|10101010|     null|null|2022-05-19|
|20202020|         |null|2022-05-19|
+--------+---------+----+----------+



In [25]:
# Adicionar una columna con la fecha y Hora  del sistema
df_personas.withColumn("fechaHora",to_timestamp(current_timestamp(),"MM-dd-yyyy HH mm ss SSS")).show(5,False)

+--------+---------+----+-----------------------+
|dni     |nombre   |edad|fechaHora              |
+--------+---------+----+-----------------------+
|99999999|Walter   |35  |2022-05-19 11:20:57.547|
|88888888|Susan    |30  |2022-05-19 11:20:57.547|
|77777777|Alejandro|12  |2022-05-19 11:20:57.547|
|66666666|Pedro    |30  |2022-05-19 11:20:57.547|
|55555555|Karina   |35  |2022-05-19 11:20:57.547|
+--------+---------+----+-----------------------+
only showing top 5 rows



In [26]:
# Capturar la fecha y hora de Inicio de ejecucion
from datetime import datetime
x=str(datetime.now())
print(x)

2022-05-19 11:20:58.164010


In [27]:
# fechas en Python
import time
print (time.strftime("%d/%m/%y"))

19/05/22


In [28]:
df_personas.withColumn('date',lit(time.strftime("%d/%m/%y"))).show()

+--------+---------+----+--------+
|     dni|   nombre|edad|    date|
+--------+---------+----+--------+
|99999999|   Walter|  35|19/05/22|
|88888888|    Susan|  30|19/05/22|
|77777777|Alejandro|  12|19/05/22|
|66666666|    Pedro|  30|19/05/22|
|55555555|   Karina|  35|19/05/22|
|44444444|     Andy|  26|19/05/22|
|33333333|   Magaly|  45|19/05/22|
|22222222|     Gian|  28|19/05/22|
|11111111|     Raul|  45|19/05/22|
|00000000|    Elena|  40|19/05/22|
|10101010|     null|null|19/05/22|
|20202020|         |null|19/05/22|
+--------+---------+----+--------+



In [29]:
# Valores maximos de cada Columna
for i in df_personas.columns:
    df_personas.select(max(length(i))).show()

+----------------+
|max(length(dni))|
+----------------+
|               8|
+----------------+

+-------------------+
|max(length(nombre))|
+-------------------+
|                  9|
+-------------------+

+-----------------+
|max(length(edad))|
+-----------------+
|                2|
+-----------------+



In [30]:
################### Join parametrizable en Funciones Pyspark

In [31]:
#Crear Dataframe Spark Cursos
schemaCursos = StructType([
    StructField("dni", StringType(),True),
    StructField("curso", StringType(),True),
    StructField("precio", DoubleType(),True)
])

dataCurso = ([("99999999", "Spark",100.50),
              ("99999999", "Scala",100.50),
              ("99999999", "Java",100.50),
              ("88888888", "Ingles",80.90),
              ("77777777", "Java",12.50)
])

df_cursos=spark.createDataFrame(dataCurso,schema=schemaCursos)
df_cursos.show()

+--------+------+------+
|     dni| curso|precio|
+--------+------+------+
|99999999| Spark| 100.5|
|99999999| Scala| 100.5|
|99999999|  Java| 100.5|
|88888888|Ingles|  80.9|
|77777777|  Java|  12.5|
+--------+------+------+



In [32]:
def join_example(dfa,dfb):
    dfa_whith_dfb=dfa.alias("a").join(dfb.alias("b"),col("a.dni")==col("b.dni"),how='inner')
    return dfa_whith_dfb

dfa_whith_dfb=join_example(df_personas,df_cursos)
dfa_whith_dfb.show()

+--------+---------+----+--------+------+------+
|     dni|   nombre|edad|     dni| curso|precio|
+--------+---------+----+--------+------+------+
|88888888|    Susan|  30|88888888|Ingles|  80.9|
|99999999|   Walter|  35|99999999| Spark| 100.5|
|99999999|   Walter|  35|99999999| Scala| 100.5|
|99999999|   Walter|  35|99999999|  Java| 100.5|
|77777777|Alejandro|  12|77777777|  Java|  12.5|
+--------+---------+----+--------+------+------+



In [33]:
################### Join parametrizable en Funciones Pandas to Pyspark

In [34]:
df_personas_pandas=df_personas.toPandas()
df_cursos_pandas=df_cursos.toPandas()

def merge_example_pandas(dfa,dfb):
    dfa_whith_dfb=dfa.merge(dfb,how='inner',on="dni")
    return dfa_whith_dfb

dfa_whith_dfb_pandas=merge_example_pandas(df_personas_pandas,df_cursos_pandas)

dfa_whith_dfb_pandas.head()

,dni,nombre,edad,curso,precio
0,99999999,Walter,35.0,Spark,100.5
1,99999999,Walter,35.0,Scala,100.5
2,99999999,Walter,35.0,Java,100.5
3,88888888,Susan,30.0,Ingles,80.9
4,77777777,Alejandro,12.0,Java,12.5


In [35]:
#Convertir a dataframe de Spark
dfa_whith_dfb_pandas_spark = spark.createDataFrame(dfa_whith_dfb_pandas)
dfa_whith_dfb_pandas_spark.show()

+--------+---------+----+------+------+
|     dni|   nombre|edad| curso|precio|
+--------+---------+----+------+------+
|99999999|   Walter|35.0| Spark| 100.5|
|99999999|   Walter|35.0| Scala| 100.5|
|99999999|   Walter|35.0|  Java| 100.5|
|88888888|    Susan|30.0|Ingles|  80.9|
|77777777|Alejandro|12.0|  Java|  12.5|
+--------+---------+----+------+------+



In [36]:
#Conteos
total=df_personas.count()

for i in df_personas.columns:
    conteo=df_personas.select(i).filter(col(i).isNotNull()).count()
    print(i,conteo,(conteo/total)*100)

dni 12 100.0
nombre 11 91.66666666666666
edad 10 83.33333333333334


In [37]:
# leer CSV para que reconozca los satos de linea
'''
df_personas = spark.read.options(header='True',inferSchema='True',delimiter=',',escape='"',quote='"',multiLine=True).\
csv(root + 'personas.csv')
'''

'\ndf_personas = spark.read.options(header=\'True\',inferSchema=\'True\',delimiter=\',\',escape=\'"\',quote=\'"\',multiLine=True).csv(root + \'personas.csv\')\n'